In [ ]:
import pandas as pd
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split

In [ ]:
rs = 7810

report_date - дата среза фичей

есть есть еденица мы не знаем какая она 1 и 2 или 3е

# change filenames here

In [ ]:
raw_data = pd.read_csv('train_dataset_Самолет.csv', low_memory=False)

In [ ]:
raw_data.info()

In [ ]:
raw_data

### text fields
colomns #

URL: 49-56 221-244 457-464 545-560 1045-1076

От куда перешли: 555 - 576

Поисковые запросы: 577 - 584

Ипотека: 129-152

Платформа 521-544

In [ ]:
list_of_text_col = ['col49','col50','col51','col52','col53','col54','col55','col56',
                            'col221','col222','col223','col224','col225','col226','col227','col228','col229','col230','col231','col232','col233','col234','col235','col236','col237','col238','col239','col240','col241','col242','col243','col244',
                            'col457','col458','col459','col460','col461','col462','col463','col464',
                            'col545','col546','col547','col548','col549','col550','col551','col552','col553','col554','col555','col556','col557','col558','col559','col560',
                            'col1045','col1046','col1047','col1048','col1049','col1050','col1051','col1052','col1053','col1054','col1055', 'col1056',
                            'col1057','col1058','col1059','col1060','col1061','col1062','col1063','col1064','col1065','col1066','col1067','col1068','col1069','col1070','col1071','col1072','col1073','col1074','col1075','col1076',
                            'col561','col562','col563','col564','col565','col566','col567','col568','col569','col570','col571','col572','col573','col574','col575','col576',
                            'col577','col578','col579','col580','col581','col582','col583','col584',
                            'col129','col130','col131','col132','col133','col134','col135','col136','col137','col138','col139','col140','col141','col142','col143','col144','col145','col146','col147','col148','col149','col150','col151','col152',
                            'col521','col522','col523','col524','col525','col526','col527','col528','col529','col530','col531','col532','col533','col534','col535','col536','col537','col538','col539','col540','col541','col542','col543','col544']
len(list_of_text_col)

In [ ]:
def extra_features(raw_data):
    EF = raw_data
    #EF = raw_data.copy()
    EF[list_of_text_col] = EF[list_of_text_col].astype(str)
    EF['text_for_search'] = EF[list_of_text_col].agg(', '.join, axis=1)
    import re
    ipoteka = ['Ипоте', 'Ipote','mortgag']
    comercial = ['comercial']
    ads = ['utm_source']
    family = ['family', 'Детска']
    parking = ['parki']
    apple = ['Mac', 'ios', 'iPhone', 'iPad', "Safari"]
    nonapple = ['Android', 'Windows', 'Linux']
    flats = ['flats']
    price_regex_min = 'price_min=\d+'
    price_regex_max = 'price_max=\d+'

    EF['ipoteka'] = 0
    EF['comercial'] = 0
    EF['ads'] = 0
    EF['family'] = 0
    EF['parking'] = 0
    EF['apple'] = 0
    EF['nonapple'] = 0
    EF['flats']=0
    EF['price_min'] = 0
    EF['price_max'] = 0
    EF['price_mid'] = 0

    for index, row in EF.iterrows():
        count = 0

        for w in ipoteka:
            if re.search(w, row['text_for_search'], re.IGNORECASE):
                count+=1
        EF.at[index, 'ipoteka'] = count
        count = 0

        for w in flats:
            if re.search(w, row['text_for_search'], re.IGNORECASE):
                count+=1
        EF.at[index, 'flats'] = count
        count = 0

        for w in apple:
            if re.search(w, row['text_for_search'], re.IGNORECASE):
                count+=1
        EF.at[index, 'apple'] = count
        count = 0

        for w in nonapple:
            if re.search(w, row['text_for_search'], re.IGNORECASE):
                count+=1
        EF.at[index, 'nonapple'] = count
        count = 0

        for w in comercial:
            if re.search(w, row['text_for_search'], re.IGNORECASE):
                count+=1
        EF.at[index, 'comercial'] = count
        count = 0

        for w in ads:
            if re.search(w, row['text_for_search'], re.IGNORECASE):
                count+=1
        EF.at[index, 'ads'] = count
        count = 0

        for w in family:
            if re.search(w, row['text_for_search'], re.IGNORECASE):
                count+=1
        EF.at[index, 'family'] = count
        count = 0

        for w in parking:
            if re.search(w, row['text_for_search'], re.IGNORECASE):
                count+=1
        EF.at[index, 'parking'] = count
        count = 0
        min_p = None
        max_p = None
        min_p = re.search(price_regex_min, row['text_for_search'])
        max_p = re.search(price_regex_max, row['text_for_search'])
    

        if min_p != None:
            min_p = min_p.group(0)[10:]
            #print(int(min_p))
            EF.at[index, 'price_min'] = int(min_p)
        if max_p != None:
            max_p = max_p.group(0)[10:]
            #print(int(max_p))
            EF.at[index, 'price_max'] = int(max_p)   
        if min_p != None and max_p != None:
            EF.at[index, 'price_mid']=(int(min_p)+int(max_p))/2
    EF = EF.drop(columns=list_of_text_col)
    EF = EF.drop(columns=['text_for_search'])
    return EF

In [ ]:
raw_data = extra_features(raw_data)

In [ ]:
raw_data

In [ ]:
raw_data['price_min'].value_counts()

In [ ]:
raw_data['price_max'].value_counts()

In [ ]:
raw_data['price_mid'].value_counts()

In [ ]:
raw_data['flats'].value_counts()

In [ ]:
raw_data['apple'].value_counts()

In [ ]:
raw_data['nonapple'].value_counts()

In [ ]:
raw_data['ipoteka'].value_counts()

In [ ]:
raw_data['ads'].value_counts()

In [ ]:
raw_data['family'].value_counts()

In [ ]:
raw_data['parking'].value_counts()

In [ ]:
raw_data['target'].value_counts()

In [ ]:
num_unq_clients = raw_data['client_id'].unique().shape[0] # Number of unque clients
num_unq_clients

In [ ]:
data = raw_data.drop(columns=['report_date','client_id'])
data

-----------

Удаление столбцов с хеш-суммами, uuid

In [ ]:
# Regexp для поиска и удаления хеш сумм, так как это уникальное значение
pattern = r'^[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{12}$'

# Функция для фильтрации колонок по регулярному выражению
def filter_columns(column_name):
    return not data[column_name].astype(str).str.match(pattern).any()

raw_data_wout_regex = data[data.columns[data.columns.to_series().apply(filter_columns)]]

In [ ]:
datat = raw_data_wout_regex.copy()
datat

--------------

Split on train test raw data

In [ ]:
datat.drop(columns=['target'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(datat.drop(columns=['target']), datat['target'], test_size=0.01, random_state=rs+20)

In [ ]:
Train_data = X_train
Train_data.insert(loc=0, column='target', value=y_train)
Test_data = X_test
Test_data.insert(loc=0, column='target', value=y_test)


In [ ]:
Train_data.shape

In [ ]:
Test_data.shape

--------------

Удаление строк с целевым признаком 0, у которых большинство (медианное значение) признаков Nan

In [ ]:
# Надо посчитать сколько в среднем заполнено значений для цп 0
median_t0 = Train_data[Train_data['target'] == 0].iloc[:,1:].apply(lambda row: row.notna().sum(), axis=1).median()
median_t0

In [ ]:
data = Train_data[~((Train_data['target'] == 0) & (Train_data.iloc[:, 1:].apply(lambda row: row.notna().sum(), axis=1) <= median_t0))]
data

------------

In [ ]:
target = data['target']
data = data.drop(['target'], axis=1)

-----

Проход окном

In [ ]:
# Дропаем колонки в которых только NaN
good_data = data.drop(data.columns[data.isna().all()], axis=1)

In [ ]:
# Серднее окном в 2 колонки
mean_gd = (good_data.count() + good_data.count().shift(-1) + good_data.count().shift(1))/3
mean_gd

In [ ]:
selcol = good_data.columns[good_data.count()>=mean_gd] ########### > || >= фичи х2 ~1300
selcol = selcol.to_list()
selcol.append(data.columns[-1]) # добавляем последнюю колонку

In [ ]:
good_data = good_data[selcol]
good_data

-------

In [ ]:
# Колонки с категориальными данными
catCols = [col for col in good_data.columns if good_data[col].dtype=="object"]
catCols[-10:-1]
#print(f'number cat columns = {len(catCols)}')

In [ ]:
categorical_columns = good_data[catCols]
numeric_columns = good_data.drop(catCols, axis=1)

-------

In [ ]:
# Нормализация данных
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
transform_gd = scaler.fit_transform(numeric_columns)

In [ ]:
Test_target = Test_data['target']
Test_data2 = Test_data[numeric_columns.columns]
Test_data3 = scaler.transform(Test_data2)

In [ ]:
HistGradientBoostingClassifier().get_params().keys()

In [ ]:
import pickle

with open ("transform_gd.pkl", 'wb') as fff:
    pickle.dump(transform_gd, file=fff)
with open ("target.pkl", 'wb') as fff:
    pickle.dump(target, file=fff)

with open ("Test_target.pkl", 'wb') as fff:
    pickle.dump(Test_target, file=fff)
with open ("Test_data3.pkl", 'wb') as fff:
    pickle.dump(Test_data3, file=fff)  



In [ ]:
with open ("transform_gd.pkl", 'rb') as fff:
    transform_gd =  pickle.load(file=fff)
with open ("target.pkl", 'rb') as fff:
    target = pickle.load(file=fff)

with open ("Test_target.pkl", 'rb') as fff:
    Test_target = pickle.load(file=fff)
with open ("Test_data3.pkl", 'rb') as fff:
    Test_data3 = pickle.load(file=fff) 

In [ ]:
from sklearn.model_selection import GridSearchCV
if False:
#parameters to tune
    parameters = {
 'max_iter': [150, 175, 200, 225, 250, 300, 350, 400],
 'learning_rate': [0.1],
 'min_samples_leaf': [1, 3, 5, 7],
 'l2_regularization': [0.05, 0.03, 0.07],
 'max_leaf_nodes': [7, 10, 13, 15],
 'max_bins': [190, 200, 210],
 'scoring': ['roc_auc'],
 'random_state' : [rs+20],
    }
#instantiate the gridsearch
    hgb_grid = GridSearchCV(HistGradientBoostingClassifier(), parameters, n_jobs=-1, 
    cv=5, scoring='roc_auc',
    verbose=3, refit=True)
#fit on the grid 
    hgb_grid.fit(transform_gd,target)
    import pickle
    with open ("hgb_grid.pkl", 'wb') as fff:
        pickle.dump(hgb_grid, file=fff)
    with open ("hgb_grid.pkl", 'rb') as fff:
        hgb_grid = pickle.load(file=fff)
    print(hgb_grid.best_params_)
    print("Test: ",roc_auc_score(Test_target, hgb_grid.best_estimator_.predict(Test_data3)))


In [ ]:

gbc= HistGradientBoostingClassifier(loss='binary_crossentropy', 
                                    max_iter=450,
                                    l2_regularization=0.05,
                                    max_leaf_nodes=15,
                                    max_depth = None,
                                    min_samples_leaf = 7, 
                                    max_bins = 200,
                                    learning_rate = 0.01,  
                                    random_state=rs+20)
gbc.fit(transform_gd,target)

print("Train: ", roc_auc_score(target, gbc.predict(transform_gd)))
print("Test: ",roc_auc_score(Test_target, gbc.predict(Test_data3)))


# change filenames here


In [ ]:
test_data = pd.read_csv('test.csv', sep=';')

In [ ]:
test_data = extra_features(test_data)

In [ ]:
test_target2 = test_data[numeric_columns.columns]
test_target3 = scaler.transform(test_target2)


In [ ]:
pred = gbc.predict_proba(test_target3)

In [ ]:
count1=0
count2=0
for i, r in pd.DataFrame(pred).iterrows():
    if r[1]<0.5:
        count1+=1
    else: 
        count2+=1
print(count1, count2) 

# change filenames here

In [ ]:
submission = pd.read_csv('submission.csv', sep=';')

In [ ]:
submission['target']=pd.DataFrame(pred)[1]

In [ ]:
submission

# change filenames here

In [ ]:
submission.to_csv('submission.csv', index=False, sep=';')